In [1]:
import copy, math, sys
import numpy as np
import itk
from itk import RTK as rtk

In [2]:
numberOfProjections = 900
firstAngle = 0.
angularArc = 180.
sid = 140 * 1000 # source to isocenter distance (mm)
sdd = 140 * 1000 + 80 # source to detector distance (mm)

pixel_spacing_in_micrometre = 1.9;
pixel_spacing_in_mm = pixel_spacing_in_micrometre * 1e-3;

theta_deg = np.linspace(0.,
                    angularArc,
                    numberOfProjections,
                    endpoint=False);

theta_rad = theta_deg / 180.0 * math.pi;

In [4]:
# Simulated Geometry
GeometryType = rtk.ThreeDCircularProjectionGeometry
geometry = GeometryType.New()

for noProj in range(0, numberOfProjections):
    angle = theta_deg[noProj]
    geometry.AddProjection(sid,
                           sdd,
                           angle);
#                            args.proj_iso_x,
#                            args.proj_iso_y,
#                            args.out_angle,
#                            args.in_angle,
#                            args.source_x,
#                            args.source_y)

geometry.SetRadiusCylindricalDetector(0)

writer = rtk.ThreeDCircularProjectionGeometryXMLFileWriter.New()
writer.SetFilename("geometry.xml")
writer.SetObject(geometry)
writer.WriteFile()

0

In [5]:
# Defines the image type
GPUImageType = rtk.Image[itk.F,3]
CPUImageType = rtk.Image[itk.F,3]

# Defines the RTK geometry object
geometry = rtk.ThreeDCircularProjectionGeometry.New()
for x in range(0,numberOfProjections):
    angle = theta_deg[noProj]
    geometry.AddProjection(sid,sdd,angle)

# Writing the geometry to disk
xmlWriter = rtk.ThreeDCircularProjectionGeometryXMLFileWriter.New()
xmlWriter.SetFilename ( "geometry.xml" )
xmlWriter.SetObject ( geometry );
xmlWriter.WriteFile();

# REIType = rtk.RayEllipsoidIntersectionImageFilter[CPUImageType, CPUImageType]
# rei = REIType.New()
# semiprincipalaxis = [ 50, 50, 50]
# center = [ 0, 0, 10]
# # Set GrayScale value, axes, center...
# rei.SetDensity(2)
# rei.SetAngle(0)
# rei.SetCenter(center)
# rei.SetAxis(semiprincipalaxis)
# rei.SetGeometry( geometry )
# rei.SetInput(constantImageSource.GetOutput())

# Read the projections
ReaderType = rtk.ImageFileReader[GPUImageType]
reader = ReaderType.New();
reader.SetFileName("projections.mha");
reader.Update();
projections = reader.GetOutput();


# Create reconstructed image
ConstantImageSourceType = rtk.ConstantImageSource[GPUImageType]
constantImageSource2 = ConstantImageSourceType.New()
sizeOutput = [ 1024, 1024, 1 ]
spacing = [ projections.GetSpacing()[0], projections.GetSpacing()[0], projections.GetSpacing()[0] ]
origin = [ -sizeOutput[0] * spacing[0] / 2.0 + spacing[0] / 2,
         -sizeOutput[1] * spacing[1] / 2.0 + spacing[1] / 2,
         -sizeOutput[2] * spacing[2] / 2.0 + spacing[2] / 2]
constantImageSource2.SetOrigin( origin )
constantImageSource2.SetSpacing( spacing )
constantImageSource2.SetSize( sizeOutput )
constantImageSource2.SetConstant(0.)

print("Origin:", constantImageSource2.GetOrigin())
print("Spacing:", constantImageSource2.GetSpacing())
print("Size:", constantImageSource2.GetSize())


# FDK reconstruction
print("Reconstructing...")
FDKGPUType = rtk.FDKConeBeamReconstructionFilter
feldkamp = FDKGPUType.New()
feldkamp.SetInput(0, constantImageSource2.GetOutput());
feldkamp.SetInput(1, projections);
feldkamp.SetGeometry(geometry);
feldkamp.GetRampFilter().SetTruncationCorrection(0.0);
feldkamp.GetRampFilter().SetHannCutFrequency(0.0);

# # Field-of-view masking
# FOVFilterType = rtk.FieldOfViewImageFilter[CPUImageType, CPUImageType]
# fieldofview = FOVFilterType.New()
# fieldofview.SetInput(0, feldkamp.GetOutput())
# fieldofview.SetProjectionsStack(rei.GetOutput())
# fieldofview.SetGeometry(geometry)

# Writer
print("Writing output image...")
WriterType = rtk.ImageFileWriter[CPUImageType]
writer = WriterType.New();
writer.SetFileName("recons.mha");
writer.SetInput(feldkamp.GetOutput()) #fieldofview.GetOutput());
writer.Update();

print("Done!")

Origin: itkPointD3 ([-0.97185, -0.97185, 0])
Spacing: itkVectorD3 ([0.0019, 0.0019, 0.0019])
Size: itkSize3 ([1024, 1024, 1])
Reconstructing...
Writing output image...
Done!
